<a href="https://colab.research.google.com/github/GOPIKA-S-S/LLM/blob/main/2348518_Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
data=pd.read_csv('/content/ner[1].csv')
data.head()

,text,labels
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


In [3]:
texts = data['text'].apply(lambda x: x.split()).values
labels = data['labels'].apply(lambda x: x.split()).values

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
label_list = list(set([label for sublist in labels for label in sublist]))
label_map = {label: i for i, label in enumerate(label_list)}

In [10]:
encoded_labels = []
for label in labels:
  encoded_labels.append([label_map[l] for l in label])

In [11]:
encoded_labels = []
for label in labels:
  encoded_labels.append([label_map[l] for l in label])

In [12]:
import numpy as np
def preprocess_data(texts, labels):
    input_ids = []
    attention_masks = []
    label_ids = []

    for i in range(len(texts)):
        # Tokenize input texts
        tokenized_input = tokenizer(texts[i], is_split_into_words=True, truncation=True, padding='max_length', max_length=128)
        input_ids.append(tokenized_input['input_ids'])
        attention_masks.append(tokenized_input['attention_mask'])

        # Align labels
        label_id = [-100] * len(tokenized_input['input_ids'])
        label_index = 0
        for j in range(1, len(tokenized_input['input_ids']) - 1):  # Skip [CLS] and [SEP]
            if tokenized_input['input_ids'][j] == tokenizer.pad_token_id:
                break
            if label_index < len(labels[i]):
                label_id[j] = labels[i][label_index]
                label_index += 1

        label_ids.append(label_id)

    return {
        'input_ids': np.array(input_ids),
        'attention_mask': np.array(attention_masks),
        'labels': np.array(label_ids)
    }

In [13]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, encoded_labels, test_size=0.2, random_state=42)
train_data = preprocess_data(train_texts, train_labels)
test_data = preprocess_data(test_texts, test_labels)

In [14]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_data['input_ids'], 'attention_mask': train_data['attention_mask']},
    train_data['labels']
)).shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': test_data['input_ids'], 'attention_mask': test_data['attention_mask']},
    test_data['labels']
)).batch(8).prefetch(tf.data.AUTOTUNE)

In [15]:
from transformers import TFBertForTokenClassification
model = TFBertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_list))

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [17]:
model.fit(train_dataset, epochs=3, validation_data=test_dataset)


Epoch 1/3
4796/4796 [==============================] - 1150s 231ms/step - loss: nan - accuracy: 3.8892e-05 - val_loss: nan - val_accuracy: 2.6878e-05
Epoch 2/3
4796/4796 [==============================] - 1102s 230ms/step - loss: nan - accuracy: 3.3598e-05 - val_loss: nan - val_accuracy: 2.6878e-05
Epoch 3/3
4796/4796 [==============================] - 1103s 230ms/step - loss: nan - accuracy: 3.3598e-05 - val_loss: nan - val_accuracy: 2.6878e-05


In [18]:
def predict_ner(text):
    tokenized_input = tokenizer(text.split(), return_tensors="tf", truncation=True, padding=True, max_length=128, is_split_into_words=True)
    outputs = model(tokenized_input)
    predictions = tf.argmax(outputs.logits, axis=-1).numpy()[0]
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'].numpy()[0])
    predicted_labels = [label_list[pred] for pred in predictions]
    return list(zip(tokens, predicted_labels))

In [19]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Load pre-trained BERT model and tokenizer
model_checkpoint = "bert-base-cased"
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=9)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_text(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    return inputs

def make_predictions(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    return logits

def map_labels_to_entities(predicted_labels, inputs):
    label_names = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"]
    entities = []
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    entity = []
    entity_type = ""
    for i, label in enumerate(predicted_labels[0]):
        if label != 0:  # Ignore "O" labels
            entity_name = label_names[label]
            token = tokens[i]
            if token.startswith("##"):
                token = token[2:]
            elif token in ["[CLS]", "[SEP]", "[PAD]"]:
                continue
            if entity_name.startswith("B-"):
                if entity:
                    entities.append((" ".join(entity), entity_type))
                    entity = []
                entity_type = entity_name[2:]
                entity.append(token)
            else:
                entity.append(token)
    if entity:
        entities.append((" ".join(entity), entity_type))
    return entities

# Test the implementation using a sample text input
text = input("Enter a text: ")
inputs = preprocess_text(text)
logits = make_predictions(inputs)
predicted_labels = torch.argmax(logits, dim=2)
entities = map_labels_to_entities(predicted_labels, inputs)

# Print the predicted entities
for entity in entities:
    print(entity)

# Example usage:
inputs = preprocess_text(text)
logits = make_predictions(inputs)
predicted_labels = torch.argmax(logits, dim=2)
entities = map_labels_to_entities(predicted_labels, inputs)

#Filter and print only 'ORG' and 'LOC' entities
for entity, entity_type in filtered_entities:
     print(entity, entity_type)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter the text: Christ university is in bangalore
('Christ university', 'ORG')
('bangalore', 'LOC')
